In [59]:
# Import dependencies
import pandas as pd
import datetime
from sqlalchemy import create_engine

# Stocks

In [86]:
stocks_url = 'https://finance.yahoo.com/most-active?offset=0&count=50'
tables = pd.read_html(stocks_url)
stock_df = tables[0]
stock_df.head()

,Symbol,Name,Price (Intraday),Change,% Change,Volume,Avg Vol (3 month),Market Cap,PE Ratio (TTM),52 Week Range
0,AMD,"Advanced Micro Devices, Inc.",27.50,-0.51,-1.82%,65.385M,67.24M,29.744B,110.44,NaN
1,BAC,Bank of America Corporation,28.40,-0.19,-0.66%,43.148M,55.623M,270.033B,10.56,NaN
2,CHK,Chesapeake Energy Corporation,2.41,-0.12,-4.74%,41.815M,46.232M,3.894B,NaN,NaN
3,ABEV,Ambev S.A.,4.05,-0.09,-2.17%,37.682M,27.914M,64.003B,23.01,NaN
4,NOK,Nokia Corporation,4.94,-0.02,-0.40%,34.757M,26.316M,27.323B,NaN,NaN


In [87]:
# Remove trailing letter and then convert datatype on the series of 'Market Cap' and 'Volume'
market_cap_cleaned = stock_df['Market Cap'].str.rstrip('B')
volume_cleaned = stock_df['Volume'].str.rstrip('M')
perc_change_cleaned = stock_df['% Change'].str.rstrip('%')

perc_change_converted = perc_change_cleaned.astype(float)
market_cap_converted = market_cap_cleaned.astype(float)
volume_converted = volume_cleaned.astype(float)

stock_df.drop(labels = '52 Week Range', axis=1, inplace=True)
stock_df.drop(labels = 'Market Cap', axis=1, inplace=True)
stock_df.drop(labels = 'Volume', axis=1, inplace=True)
stock_df.drop(labels = '% Change', axis=1, inplace=True)
stock_df['Market Cap ($Bil.)'] = market_cap_converted
stock_df['Volume (Mil.)'] = volume_converted
stock_df['Percentage_change'] = perc_change_converted

In [88]:
stock_df['Time'] = datetime.datetime.now()
stock_df_filtered = stock_df[stock_df['Market Cap ($Bil.)'] > 100.0]

stock_df_filtered.reset_index(inplace=True)
stock_df_filtered.drop(labels = 'index', axis=1, inplace=True)

stock_df_filtered.head()

,Symbol,Name,Price (Intraday),Change,Avg Vol (3 month),PE Ratio (TTM),Market Cap ($Bil.),Volume (Mil.),Percentage_change,Time
0,BAC,Bank of America Corporation,28.40,-0.19,55.623M,10.56,270.033,43.148,-0.66,2019-05-18 12:07:32.163107
1,AAPL,Apple Inc.,189.00,-1.08,28.748M,15.90,869.604,32.879,-0.57,2019-05-18 12:07:32.163107
2,T,AT&T Inc.,31.80,0.18,29.111M,11.99,232.076,27.936,0.57,2019-05-18 12:07:32.163107
3,PFE,Pfizer Inc.,41.47,-0.17,23.36M,21.30,230.233,22.372,-0.41,2019-05-18 12:07:32.163107
4,MSFT,Microsoft Corporation,128.07,-0.86,25.084M,28.45,981.377,25.771,-0.67,2019-05-18 12:07:32.163107


# ETFs

In [89]:
etfs_url = 'https://finance.yahoo.com/etfs?offset=0&count=50'
tables = pd.read_html(etfs_url)
etfs_df = tables[0]
etfs_df.head()

,Symbol,Name,Price (Intraday),Change,% Change,Volume,50 Day Average,200 Day Average,52 Week Range
0,INCO,Columbia India Consumer ETF,39.89,0.68,+1.75%,5101,41.38,41.73,NaN
1,GOEX,Global X Gold Explorers ETF,19.42,0.16,+0.80%,3711,20.14,20.22,NaN
2,YMLI,VanEck Vectors High Income Infrastructure MLP ETF,12.22,0.02,+0.16%,2452,12.24,12.09,NaN
3,ZROZ,PIMCO 25+ Year Zero Coupon U.S. Treasury Index...,118.94,0.74,+0.63%,12589,115.96,111.81,NaN
4,IDU,iShares U.S. Utilities ETF,148.81,0.77,+0.52%,16637,146.56,141.62,NaN


In [90]:
etf_perc_change_cleaned = etfs_df['% Change'].str.rstrip('%')

etf_perc_change_converted = etf_perc_change_cleaned.astype(float)

etfs_df.drop(labels = '52 Week Range', axis=1, inplace=True)
etfs_df.drop(labels = '% Change', axis=1, inplace=True)

etfs_df['Percentage_change'] = etf_perc_change_converted
etfs_df['Time'] = datetime.datetime.now()
etfs_df.head()

,Symbol,Name,Price (Intraday),Change,Volume,50 Day Average,200 Day Average,Percentage_change,Time
0,INCO,Columbia India Consumer ETF,39.89,0.68,5101,41.38,41.73,1.75,2019-05-18 12:07:38.985497
1,GOEX,Global X Gold Explorers ETF,19.42,0.16,3711,20.14,20.22,0.80,2019-05-18 12:07:38.985497
2,YMLI,VanEck Vectors High Income Infrastructure MLP ETF,12.22,0.02,2452,12.24,12.09,0.16,2019-05-18 12:07:38.985497
3,ZROZ,PIMCO 25+ Year Zero Coupon U.S. Treasury Index...,118.94,0.74,12589,115.96,111.81,0.63,2019-05-18 12:07:38.985497
4,IDU,iShares U.S. Utilities ETF,148.81,0.77,16637,146.56,141.62,0.52,2019-05-18 12:07:38.985497


# Load

In [91]:
rds_connection_string = "root:Password@localhost"
engine = create_engine(f'mysql://{rds_connection_string}')

In [92]:
engine.execute('CREATE DATABASE IF NOT EXISTS wolves_db')

In [93]:
engine.execute('USE wolves_db')

In [94]:
stock_df_filtered.to_sql(name='market_data', con=engine, if_exists='append', index=False)

In [95]:
etfs_df.to_sql(name='etfs_data', con=engine, if_exists='append', index=False)